Analys över polishändelser i Sverige

In [1]:
%config IPCompleter.greedy=True
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
from sqlalchemy import create_engine, MetaData, Table

In [2]:
url = 'https://polisen.se/api/events'                  # Gets the 500 latest events
#url = 'https://polisen.se/api/events?DateTime=2019-01' # Gets the event for january 2019 (500 items max)
response = requests.get(url)
print(type(response))
rjs = response.json()   # Save the response in json format for easy parsing, but this is a list containing dicts

<class 'requests.models.Response'>


In [3]:
idl = []
datetimel = []
namel = []
summaryl = []
urll = []
typel = []
location_namel = []
longitudel = []
latitudel = []
for element in rjs:
    idl.append(element['id'])
    datetimel.append(element['datetime'])
    namel.append(element['name'])
    summaryl.append(element['summary'])
    urll.append(element['url'])
    typel.append(element['type'])
    location_namel.append(element['location']['name'])
    latlong_temp = element['location']['gps'].split(',')
    longitudel.append(latlong_temp[0])
    latitudel.append(latlong_temp[1])

In [4]:
df = pd.DataFrame({'id': idl,
                  'datetime': datetimel,
                  'name': namel,
                  'summary': summaryl,
                  'url': urll,
                  'type': typel,
                  'location': location_namel,
                  'longitude': longitudel,
                  'latitude': latitudel})

In [5]:
df.head()

,id,datetime,name,summary,url,type,location,longitude,latitude
0,172409,2019-11-14 9:00:30 +01:00,"14 november 09:00, Misshandel, Ludvika",En misshandel har skett i Grängesberg.,https://polisen.se/aktuellt/handelser/2019/nov...,Misshandel,Ludvika,60.152358,15.191639
1,172416,2019-11-14 10:49:20 +01:00,"14 november 10:49, Trafikolycka, Norrköping",Cyklist påkörd i centrala Norrköping.,https://polisen.se/aktuellt/handelser/2019/nov...,Trafikolycka,Norrköping,58.587745,16.192421
2,172407,2019-11-14 9:38:00 +01:00,"14 november 09:38, Trafikolycka, Jönköping",En lastbil med släp har vält på riksväg 26 i Ryd.,https://polisen.se/aktuellt/handelser/2019/nov...,Trafikolycka,Jönköping,57.782614,14.161788
3,172414,2019-11-14 10:43:53 +01:00,"14 november 10:43, Trafikolycka, Norrköping",Krock mellan bilar på riksväg 51 vid Svärtinge.,https://polisen.se/aktuellt/handelser/2019/nov...,Trafikolycka,Norrköping,58.587745,16.192421
4,172279,2019-11-13 8:28:04 +01:00,"13 november 08:28, Rån, Håbo",En ung man har rånats på alla sina kläder. En ...,https://polisen.se/aktuellt/handelser/2019/nov...,Rån,Håbo,59.627071,17.452298


In [6]:
print('earliest: ' + str(df.datetime.min()) + " and latest:" + str(df.datetime.max()))

earliest: 2019-11-03 13:29:04 +01:00 and latest:2019-11-14 9:50:55 +01:00


In [7]:
print("number of unique categories:  " + str(len(df.type.unique())))
print(df.type.value_counts().head())

number of unique categories:  57
Trafikolycka           98
Övrigt                 53
Trafikolycka, vilt     37
Rattfylleri            23
Sammanfattning natt    22
Name: type, dtype: int64


In [8]:
import folium

Create a map with markers corresponding to all event locations

In [9]:
# Make an empty map
m = folium.Map(location=[20, 0], tiles="OpenStreetMap", zoom_start=2)
 
# I can add marker one by one on the map
for i in range(0,len(df)):
    folium.Marker([df.iloc[i]['longitude'], df.iloc[i]['latitude']], popup=df.iloc[i]['location']).add_to(m)
 
# Save it as html
m.save('312_markers_on_folium_map1.html')
m

Create a map with density depending on the report type

In [10]:
# Create a dataframe only containing trafic accidents
ta = df[df['type'].isin(['Trafikolycka', 'Trafikolycka, vilt'])]
oc_dic = dict()
for place in ta.location:
    if place in oc_dic.keys():
        oc_dic[place] += 1
    else:
        oc_dic[place] = 1

In [11]:
# Make an empty map
m = folium.Map(location=[59.334591,18.063240], tiles="OpenStreetMap", zoom_start=2)
 
# Add markers with size based on number of occurances?
for i in range(0,len(ta)):
   folium.Circle(
      location=[ta.iloc[i]['longitude'], ta.iloc[i]['latitude']],
      popup=ta.iloc[i]['location'],
      radius=oc_dic[ta.iloc[i]['location']]*10000,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(m)
 
# Save it as html
m.save('mymap.html')
m

Let's investigate bomb threats and detonations

In [25]:
boom = df[df['type'].isin(['Detonation', 'Bombhot'])]

In [26]:
boom.head()

,id,datetime,name,summary,url,type,location,longitude,latitude
186,172010,2019-11-11 23:57:20 +01:00,"11 november 23:57, Detonation, Hässleholm","Detonation i anslutning till garage, Sösdala.",https://polisen.se/aktuellt/handelser/2019/nov...,Detonation,Hässleholm,56.158915,13.766765
323,171835,2019-11-11 1:25:04 +01:00,"11 november 01:25, Detonation, Helsingborg",Detonation på Östra Sandgatan i Helsingborg.,https://polisen.se/aktuellt/handelser/2019/nov...,Detonation,Helsingborg,56.046467,12.694512
433,171720,2019-11-09 20:57:52 +01:00,"09 november 20:57, Detonation, Malmö","Larm om hög smäll, Hästhagen.",https://polisen.se/aktuellt/handelser/2019/nov...,Detonation,Malmö,55.604981,13.003822


In [32]:
sthlm = df[df['location'] == 'Stockholm']

In [33]:
sthlm

,id,datetime,name,summary,url,type,location,longitude,latitude
1,172270,2019-11-13 13:16:05 +01:00,"13 november 13:16, Stöld, Stockholm",En firma som kör ut tv-apparater har vid en le...,https://polisen.se/aktuellt/handelser/2019/nov...,Stöld,Stockholm,59.329324,18.068581
4,172267,2019-11-13 13:09:01 +01:00,"13 november 13:09, Trafikbrott, Stockholm",SOS Alarm meddelar polisen att en bil har kört...,https://polisen.se/aktuellt/handelser/2019/nov...,Trafikbrott,Stockholm,59.329324,18.068581
71,172147,2019-11-12 17:15:03 +01:00,"12 november 17:15, Häleri, Stockholm",Polisen fick samtal om att en man tagit sig in...,https://polisen.se/aktuellt/handelser/2019/nov...,Häleri,Stockholm,59.329324,18.068581
94,172150,2019-11-12 18:24:03 +01:00,"12 november 18:24, Mord/dråp, Stockholm",Polisen kan inte utesluta att den man som antr...,https://polisen.se/aktuellt/handelser/2019/nov...,Mord/dråp,Stockholm,59.329324,18.068581
108,172140,2019-11-12 17:38:09 +01:00,"12 november 17:38, Stöld, Stockholm",Polisen har identifierat den man som är misstä...,https://polisen.se/aktuellt/handelser/2019/nov...,Stöld,Stockholm,59.329324,18.068581
113,172136,2019-11-12 15:37:02 +01:00,"12 november 15:37, Narkotikabrott, Stockholm",På Sergels torg grep ordningsvakter en man mis...,https://polisen.se/aktuellt/handelser/2019/nov...,Narkotikabrott,Stockholm,59.329324,18.068581
115,172135,2019-11-12 15:06:02 +01:00,"12 november 15:06, Övrigt, Stockholm",Nio personer greps av personal i riksdagshuset...,https://polisen.se/aktuellt/handelser/2019/nov...,Övrigt,Stockholm,59.329324,18.068581
144,172078,2019-11-12 12:23:17 +01:00,"12 november 12:23, Bråk, Stockholm",Det har varit bråk på en skola i Alvik vilket ...,https://polisen.se/aktuellt/handelser/2019/nov...,Bråk,Stockholm,59.329324,18.068581
216,171978,2019-11-11 16:02:37 +01:00,"11 november 16:02, Hemfridsbrott, Stockholm",Anmälan upprättad efter att man uppgivit att h...,https://polisen.se/aktuellt/handelser/2019/nov...,Hemfridsbrott,Stockholm,59.329324,18.068581
234,171958,2019-11-11 17:47:07 +01:00,"11 november 17:47, Trafikolycka, Stockholm",Trafikolycka på E18 i höjd med Kista.,https://polisen.se/aktuellt/handelser/2019/nov...,Trafikolycka,Stockholm,59.329324,18.068581


In [35]:
print(sthlm.iloc[0].summary)
print(sthlm.iloc[1].summary)
print(sthlm.iloc[2].summary)

En firma som kör ut tv-apparater har vid en leverans av en tv på Södermalm blivit utsatta för en stöld från den lastbil de använder sig av.
SOS Alarm meddelar polisen att en bil har kört igenom ett staket i Solberga. Någon ska dessutom samtidigt observerat en hög knall.
Polisen fick samtal om att en man tagit sig in i ett garage på Sankt Göransgatan
